(context_windows_tutorial)=
# Working with Context Windows

> _Author:_ Pietro Novelli — [@pie_novelli](https://twitter.com/pie_novelli)

Kooplearn's fundamental data units are [context windows](kooplearn_data_paradigm). In the practice of dynamical system analysis, and more generally sequence modelling, it is arguably more natural to conceive a "data point" as a context window containing the dynamical information at a given point in time. A context window encloses the 'past' in its _lookback window_ and the 'future' in its _lookforward window_. Intuitively, everything in the lookback window is the information we need to provide, at inference time, to predict what will happen in the lookforward window. By using context windows we depart from the usual paradigm in supervised learning in which data is categorized into inputs and outputs.

As of version 1.1.0, Kooplearn exposes the following key objects:

- {class}`kooplearn.abc.ContextWindow`: the root class defining generic context windows.
- {class}`kooplearn.abc.ContextWindowDataset`: representing generic collections of context windows.
- {class}`kooplearn.data.TensorContextDataset`: representing collections of context windows with tensor elements.
- {class}`kooplearn.data.TrajectoryContextDataset`: representing collections of context windows obtained obtained sliding through a single long trajectory.

In this hands-on tutorial notebook we discuss each one of these objects and their specific use-cases.

## Generic context windows: {class}`kooplearn.abc.ContextWindow`

{class}`kooplearn.abc.ContextWindow` is just a subclass of [`Sequence`](https://docs.python.org/3/library/collections.abc.html#collections.abc.Sequence). For example, in language modelling [context windows are usually made of sequence of characters or tokens](https://en.wikipedia.org/wiki/Large_language_model#Prompt_engineering,_attention_mechanism,_and_context_window):

In [1]:
from kooplearn.abc import ContextWindow

text = "the quick brown fox"
# Replace blank spaces with dashes
text = text.replace(' ', '_')

context_window = ContextWindow([char for char in text])
print(context_window)

ContextWindow <context_length=19, data=['t', 'h', 'e', '_', 'q', 'u', 'i', 'c', 'k', '_', 'b', 'r', 'o', 'w', 'n', '_', 'f', 'o', 'x']>


the {class}`kooplearn.abc.ContextWindow` object has the {class}`kooplearn.abc.ContextWindow.context_length` attribute defined upon initialization. {class}`kooplearn.abc.ContextWindow.context_length` is inherited by _all the subclasses of_ {class}`kooplearn.abc.ContextWindow`.

In [2]:
context_window.context_length

19

To easily access the data in `context_window`, {class}`kooplearn.abc.ContextWindow` exposes three key methods:

- {class}`kooplearn.abc.ContextWindow.lookback`
- {class}`kooplearn.abc.ContextWindow.lookforward`
- {class}`kooplearn.abc.ContextWindow.slice`

The length of the lookback window is _not_ an attribute of {class}`kooplearn.abc.ContextWindow`, as it heavily depend to the model. Therefore, in kooplearn the lookback length is defined independently from each model and stored in the attribute {class}`kooplearn.abc.BaseModel.lookback_len`. For this example, let's just fix the lookback length to 10. The `lookback` and `lookforward` methods are extremely straightforward: given a lookback length return a sequence with the lookback or lookforward window, respectively.

In [3]:
lookback_length = 10

lb_window = context_window.lookback(lookback_length)

# The lookforward window length is inferenced from lookback_length directly
lf_window = context_window.lookforward(lookback_length)

print(f"Lookback window: {lb_window}")
print(f"Lookforward window: {lf_window}")

Lookback window: ['t', 'h', 'e', '_', 'q', 'u', 'i', 'c', 'k', '_']
Lookforward window: ['b', 'r', 'o', 'w', 'n', '_', 'f', 'o', 'x']


The `lookback` method also accepts an optional `slide_by` keyword argument, which can be used to return lookback windows shifted by an offset.

In [4]:
# Skip the first 4 elements
shifted_lb_window = context_window.lookback(lookback_length, slide_by=4) 

print(f"Shifted lookback window: {shifted_lb_window}") 

# Of course, if slide_by is too high an error is raised
try:
    # lookback length of 10 + sliding of 15 > context length of 19
    overflow_lb_window = context_window.lookback(lookback_length, slide_by=15)
    print(f"Shifted lookback window: {overflow_lb_window}") 
except ValueError as e:
    print(f"ValueError: {e}")

Shifted lookback window: ['q', 'u', 'i', 'c', 'k', '_', 'b', 'r', 'o', 'w']
ValueError: Invalid slide_by = 15 for lookback_length = 10 and Context of length = 19. It should be 0 <= slide_by <= context_length - lookback_length


Finally, {class}`kooplearn.abc.ContextWindow.slice` is used to get arbitrary slices of the context window through [python slice objects](https://docs.python.org/3/glossary.html#term-slice)

In [5]:
slice_obj = slice(2, 16, 2) # (start, stop, step) 
context_slice = context_window.slice(slice_obj)
print(f"Arbitrary slice: {context_slice}")

Arbitrary slice: ['e', 'q', 'i', 'k', 'b', 'o', 'n']


## Collections of context windows: {class}`kooplearn.abc.ContextWindowDataset`

In real applications we always deal with _collections_ of context windows. For this purpose, kooplearn exposes {class}`kooplearn.abc.ContextWindowDataset`, inheriting every method of {class}`kooplearn.abc.ContextWindow`. A Context Window Dataset can contain heterogeneous data. The only requirement is that every context window in the collection should have the same context length.

In [6]:
from kooplearn.abc import ContextWindowDataset
 
raw_contexts = [
    [1, 2, 3, 4, 5],
    ['a', 'b', 'c', 'd', 'e'],
    ['a', 1.0, True, 'd', 5]
]

contexts = ContextWindowDataset(raw_contexts)
print(contexts)

ContextWindowDataset <item_count=3, context_length=5, data=[[1, 2, 3, 4, 5], ['a', 'b', 'c', 'd', 'e'], ['a', 1.0, True, 'd', 5]]>


And of course every slicing method works out of the box.

In [7]:
lookback_length = 2

lb_window = contexts.lookback(lookback_length)
lf_window = contexts.lookforward(lookback_length)
shifted_lb_window = contexts.lookback(lookback_length, slide_by=2)

slice_obj = slice(1, 4, 1) # (start, stop, step) 
context_slice = contexts.slice(slice_obj)


print(f"Lookback window: {lb_window}")
print(f"Lookforward window: {lf_window}")
print(f"Shifted lookback window: {shifted_lb_window}") 
print(f"Arbitrary slice: {context_slice}")

Lookback window: [[1, 2], ['a', 'b'], ['a', 1.0]]
Lookforward window: [[3, 4, 5], ['c', 'd', 'e'], [True, 'd', 5]]
Shifted lookback window: [[3, 4], ['c', 'd'], [True, 'd']]
Arbitrary slice: [[2, 3, 4], ['b', 'c', 'd'], [1.0, True, 'd']]


{class}`kooplearn.abc.ContextWindowDataset` can also be iterated:

In [8]:
for ctx in contexts:
    print(ctx)

ContextWindow <context_length=5, data=[1, 2, 3, 4, 5]>
ContextWindow <context_length=5, data=['a', 'b', 'c', 'd', 'e']>
ContextWindow <context_length=5, data=['a', 1.0, True, 'd', 5]>


### Tensor Context Windows: {class}`kooplearn.data.TensorContextDataset`

While {class}`kooplearn.abc.ContextWindowDataset` can operate with eterogeneous data it is usually the case that states, that is elements in the context windows, are just multidimensional arrays of features. In kooplearn we implemented {class}`kooplearn.data.TensorContextDataset` to efficiently operate and slice through tensorial data. `TensorContextDataset` operate seamlessly with either `torch` and `numpy` tensors, and are initialized by providing a tensor of shape `(num_context_windows, context_length, *feature_shape)`.

In [9]:
import numpy as np
import torch
from kooplearn.data import TensorContextDataset

# A collection of 3 context windows, with context length of 5 and 2x2 feature shape
num_context_windows = 3
context_length = 5
feature_shape = (2,2)

rng = np.random.default_rng(seed = 0) # for Reproducibility 

np_raw_contexts = rng.random((num_context_windows, context_length, *feature_shape))

np_contexts = TensorContextDataset(np_raw_contexts)

print(np_contexts)

TensorContextDataset <item_count=3, context_length=5, data=[[[[0.63696169 0.26978671]
   [0.04097352 0.01652764]]

  [[0.81327024 0.91275558]
   [0.60663578 0.72949656]]

  [[0.54362499 0.93507242]
   [0.81585355 0.0027385 ]]

  [[0.85740428 0.03358558]
   [0.72965545 0.17565562]]

  [[0.86317892 0.54146122]
   [0.29971189 0.42268722]]]


 [[[0.02831967 0.12428328]
   [0.67062441 0.64718951]]

  [[0.61538511 0.38367755]
   [0.99720994 0.98083534]]

  [[0.68554198 0.65045928]
   [0.68844673 0.38892142]]

  [[0.13509651 0.72148834]
   [0.52535432 0.31024188]]

  [[0.48583536 0.88948783]
   [0.93404352 0.3577952 ]]]


 [[[0.57152983 0.32186939]
   [0.59430003 0.33791123]]

  [[0.391619   0.89027435]
   [0.22715759 0.62318714]]

  [[0.08401534 0.83264415]
   [0.78709831 0.23936944]]

  [[0.87648423 0.05856803]
   [0.33611706 0.15027947]]

  [[0.45033937 0.79632427]
   [0.23064221 0.0520213 ]]]]>


{class}`kooplearn.data.TensorContextDataset` can be initialized by setting a `backend`, which can be either `torch` or `numpy` and any backend keyword argument such as dtype or device. For example we can initialize a `torch`-backed context dataset with complex dtype as:

In [10]:
complex_torch_contexts = TensorContextDataset(np_raw_contexts, backend='torch', dtype=torch.cfloat)

print(f"The original data is of type {type(np_raw_contexts)}")
print(f"The data in complex_torch_contexts is of type {type(complex_torch_contexts.data)}")

print(f"The original dtype is {np_raw_contexts.dtype}")
print(f"The data in complex_torch_contexts is of dtype {complex_torch_contexts.data.dtype}")


The original data is of type <class 'numpy.ndarray'>
The data in complex_torch_contexts is of type <class 'torch.Tensor'>
The original dtype is float64
The data in complex_torch_contexts is of dtype torch.complex64


### Context Windows from Trajectories: {class}`kooplearn.data.TrajectoryContextDataset`

Finally, when we have a long trajectory of points we can construct a collection of context windows via {class}`kooplearn.data.TrajectoryContextDataset` or via {func}`kooplearn.data.traj_to_contexts`. 

In [11]:
from kooplearn.data import TrajectoryContextDataset

long_trajectory = np.arange(6, dtype = np.float32)
print(long_trajectory)

contexts = TrajectoryContextDataset(long_trajectory, context_length = 4)
print(contexts)


[0. 1. 2. 3. 4. 5.]
TrajectoryContextDataset <item_count=3, context_length=4, data=[[[0.]
  [1.]
  [2.]
  [3.]]

 [[1.]
  [2.]
  [3.]
  [4.]]

 [[2.]
  [3.]
  [4.]
  [5.]]]>


Of course every slicing operation still works:

In [12]:
lookback_length = 2

lb_window = contexts.lookback(lookback_length)
lf_window = contexts.lookforward(lookback_length)
shifted_lb_window = contexts.lookback(lookback_length, slide_by=2)

slice_obj = slice(1, 4, 1) # (start, stop, step) 
context_slice = contexts.slice(slice_obj)


print(f"Lookback window: {lb_window}")
print(f"Lookforward window: {lf_window}")
print(f"Shifted lookback window: {shifted_lb_window}") 
print(f"Arbitrary slice: {context_slice}")

Lookback window: [[[0.]
  [1.]]

 [[1.]
  [2.]]

 [[2.]
  [3.]]]
Lookforward window: [[[2.]
  [3.]]

 [[3.]
  [4.]]

 [[4.]
  [5.]]]
Shifted lookback window: [[[2.]
  [3.]]

 [[3.]
  [4.]]

 [[4.]
  [5.]]]
Arbitrary slice: [[[1.]
  [2.]
  [3.]]

 [[2.]
  [3.]
  [4.]]

 [[3.]
  [4.]
  [5.]]]
